In [39]:
# Dependencies
import pandas as pd
import numpy as np

In [40]:
# Create df
df = pd.read_csv("Resources/purchase_data.csv")
df = df.rename(columns={
    "Purchase ID":"purchase_id",
    "SN":"sn",
    "Age":"age",
    "Gender":"gender",
    "Item ID":"item_id",
    "Item Name":"item_name",
    "Price":"price"
})
df.head()

,purchase_id,sn,age,gender,item_id,item_name,price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [41]:
### Player Count
tot_players = df.sn.unique().shape[0]

player_df = pd.DataFrame({"Total Players":tot_players}, index=[0])
player_df

,Total Players
0,576


In [42]:
### Purchasing Analysis (Total)
unique_items = df.item_id.unique().shape[0]
avg_price = round(df.price.mean(),2)
tot_purchases = df.shape[0]
tot_revenue = df.price.sum()

purchase_analysis = pd.DataFrame({
    "Unique Items": unique_items,
    "Average Price": avg_price,
    "Number of Purchases": tot_purchases,
    "Total Revenue": tot_revenue,
}, index=[0])
purchase_analysis

,Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,3.05,780,2379.77


In [43]:
### Gender Demographics
gender_group = df.groupby(['gender'])
gender_counts = (gender_group['sn'].unique()).str.len()
gender_total = gender_counts.sum(axis = 0, skipna = True)

gender_demographics = pd.DataFrame({
    "Total Count":gender_counts[:],
    "Percentage of Players":round(((gender_counts[:]/gender_total)*100),2),
})
gender_demographics

,Total Count,Percentage of Players
gender,,
Female,81,14.06
Male,484,84.03
Other / Non-Disclosed,11,1.91


In [44]:
### Purchasing Analysis (Gender)
purchasing_analysis_df = pd.DataFrame({
    'Genders':[i][0],
    'Purchase Count':[df[df.gender == i].shape[0]][0], 
    'Avg. Purchase Price':round([df[df.gender == i].price.mean()][0],2),
    'Tot. Purchase Value':[df[df.gender == i].price.sum()][0],
    'Avg. Total':round([df[df.gender == i].price.sum()][0]/gender_counts[i],2),
} for i in gender_counts.index)
purchasing_analysis_df.set_index(['Genders'])

,Purchase Count,Avg. Purchase Price,Tot. Purchase Value,Avg. Total
Genders,,,,
Female,113,3.20,361.94,4.47
Male,652,3.02,1967.64,4.07
Other / Non-Disclosed,15,3.35,50.19,4.56


In [45]:
### Age Demographics
age_bins = [0]+[10+5*i for i in range(7)]+[100]
upper = age_bins[1:-1]
lower = age_bins[:-2]
age_labels = ["{0}-{1}".format(l, u-1) for l,u in zip(lower, upper)]+["40+"]
df['ages'] = pd.cut(df.age,age_bins,right=False,labels=age_labels)
ages_groups = df.groupby("ages")
ages_counts = ages_groups['sn'].unique().map(len)

ages_df = pd.DataFrame({
    "Age Counts":ages_counts,
    "Pct. of Total":round(ages_counts/ages_counts.sum()*100,2),
})
ages_df

,Age Counts,Pct. of Total
ages,,
0-9,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


In [46]:
### Age Demographics
age_demo_counts = ages_groups['sn'].size()
age_demo_average = ages_groups['price'].mean()
age_demo_total = ages_groups['price'].sum()

age_demos_df = pd.DataFrame({
    "Age Counts":age_demo_counts,
    "Avg. Purchase Price":round(age_demo_average,2),
    "Tot. Purchase Value":age_demo_total,
    "Avg Total Purchase per Person":round(age_demo_total/ages_counts,2),
})
age_demos_df

,Age Counts,Avg. Purchase Price,Tot. Purchase Value,Avg Total Purchase per Person
ages,,,,
0-9,23,3.35,77.13,4.54
10-14,28,2.96,82.78,3.76
15-19,136,3.04,412.89,3.86
20-24,365,3.05,1114.06,4.32
25-29,101,2.90,293.00,3.81
30-34,73,2.93,214.00,4.12
35-39,41,3.60,147.67,4.76
40+,13,2.94,38.24,3.19


In [12]:
### Top Spenders
sn_groups = df.groupby(['sn'])
sn_count = sn_groups['sn'].count()
sn_average = sn_groups['price'].mean()
sn_total = sn_groups["price"].sum()

sn_df = pd.DataFrame({
    "Purchase Count":sn_count,
    "Avg. Purchase Price":round(sn_average,2),
    "Tot. Purchase Value":sn_total,
})

sn_df.sort_values(by=['Tot. Purchase Value'], ascending=False).head()

,Purchase Count,Avg. Purchase Price,Tot. Purchase Value
sn,,,
Lisosia93,5,3.79,18.96
Idastidru52,4,3.86,15.45
Chamjask73,3,4.61,13.83
Iral74,4,3.40,13.62
Iskadarya95,3,4.37,13.10


In [13]:
### Most Popular Items
item_groups = df.groupby(['item_id'])
item_name = item_groups.item_name.first()
item_count = item_groups.item_id.count()
item_price = item_groups.price.first()
item_total = item_groups.price.sum()

item_df = pd.DataFrame({
    "Item Name": item_name,
    "Purchase Count": item_count,
    "Item Price": item_price,
    "Tot. Purchase Value": item_total,
})

item_df.sort_values(by=['Purchase Count'], ascending=False).head()

,Item Name,Purchase Count,Item Price,Tot. Purchase Value
item_id,,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
145,Fiery Glass Crusader,9,4.58,41.22
108,"Extraction, Quickblade Of Trembling Hands",9,3.53,31.77
82,Nirvana,9,4.90,44.10
19,"Pursuit, Cudgel of Necromancy",8,1.02,8.16


In [14]:
### Most Profitable Items
item_df.sort_values(by=['Tot. Purchase Value'], ascending=False).head()

,Item Name,Purchase Count,Item Price,Tot. Purchase Value
item_id,,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,50.76
82,Nirvana,9,4.90,44.10
145,Fiery Glass Crusader,9,4.58,41.22
92,Final Critic,8,4.88,39.04
103,Singed Scalpel,8,4.35,34.80
